In [14]:
import time

import torch
import torch.nn as nn
from torch import nn, optim, Tensor
from torch.utils.data import DataLoader
import torch.nn.functional as F

import numpy as np
from typing import List, Tuple, Any, Optional, Dict

from data.raw_data import Instance, Data
from data.data_utils import Dictionary, triples_to_indices, collate
from data.dataset import CLUTRRdata
from models.encoders import EncoderLSTM
from models.decoders import DecoderLSTM
from training.train import train, predict_raw, evaluate

In [2]:
test_data_path = ['data/clutrr-data/data_089907f8/1.2_test.csv',
                  'data/clutrr-data/data_089907f8/1.3_test.csv',
                 'data/clutrr-data/data_089907f8/1.4_test.csv']
data = Data(test_paths = test_data_path)
dictionary = Dictionary(data)
story, target = data.train[5080].story, data.train[5000].target
s, p, o = target
print(f'Number of entities {dictionary.num_entities}')
print(f'Example story: {story}')
print(f'Example target: {target}')

entity_embeddings = nn.Embedding(dictionary.num_entities, 100, sparse=True)
relation_embeddings = nn.Embedding(dictionary.num_relations, 100, sparse=True)

# Currently no backprop through embeddings
entity_embeddings.weight.requires_grad = False
relation_embeddings.weight.requires_grad = False
print(f'entity embeddings shape: {entity_embeddings.weight.shape}')

indices, ent, rel = triples_to_indices(dictionary, story)

Number of entities 104
Example story: [('Dwight', 'brother', 'Christopher'), ('Christopher', 'daughter', 'Lucille')]
Example target: ('James', 'father', 'Jason')
entity embeddings shape: torch.Size([104, 100])


In [3]:
train_set = CLUTRRdata(data.train, dictionary)
test_set = CLUTRRdata(data.test, dictionary)

In [4]:
encoder = EncoderLSTM(hidden_size = 100,
                    entity_embeddings = entity_embeddings,
                    relation_embeddings = relation_embeddings)

decoder = DecoderLSTM(hidden_size = 100,
                    entity_embeddings = entity_embeddings,
                    num_relations = dictionary.num_relations)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder = encoder.to(device)
decoder = decoder.to(device)

In [17]:
train(encoder,
      decoder,
      train_set,
      test_set,
      epochs = 2,
      learning_rate = 0.01)

Training model...
Initial values: 
Train loss: 1.32 | Train acc: 99.24% | Test loss: 112.39 | Test acc: 62.76%
Epoch [1/2],Train loss: 1.7048,Test loss: 155.0830 Train acc: 99.07% Test acc: 59.16%
Epoch [2/2],Train loss: 0.9743,Test loss: 173.1465 Train acc: 99.40% Test acc: 53.45%
Finished training



In [7]:
test_data_path = ['data/clutrr-data/data_089907f8/1.4_test.csv']
data2 = Data(test_paths = test_data_path)
#dictionary = Dictionary(data2)

In [8]:
for i in range(10):
    print(predict_raw(data2.test[i], encoder, decoder, dictionary))

Story: [('Jenny', 'daughter', 'Anne'), ('Anne', 'father', 'Richard'), ('Richard', 'daughter', 'Kimberley'), ('Kimberley', 'uncle', 'Benjamin')]
Target: ('Jenny', 'brother', 'Benjamin')
Predicted relation: brother
None
Story: [('Cornelius', 'daughter', 'April'), ('April', 'brother', 'Frank'), ('Frank', 'daughter', 'Julia'), ('Julia', 'sister', 'Beverly')]
Target: ('Cornelius', 'granddaughter', 'Beverly')
Predicted relation: niece
None
Story: [('Christine', 'sister', 'Connie'), ('Connie', 'mother', 'Patricia'), ('Patricia', 'daughter', 'Connie'), ('Connie', 'uncle', 'Frank')]
Target: ('Christine', 'uncle', 'Frank')
Predicted relation: brother
None
Story: [('Frank', 'son', 'Charles'), ('Charles', 'sister', 'Rachel'), ('Rachel', 'sister', 'Beverly'), ('Beverly', 'uncle', 'Richard')]
Target: ('Frank', 'brother', 'Richard')
Predicted relation: uncle
None
Story: [('Jenny', 'daughter', 'Kimberley'), ('Kimberley', 'father', 'Richard'), ('Richard', 'sister', 'Patricia'), ('Patricia', 'daughter',

In [23]:
test_data_path = ['data/clutrr-data/data_089907f8/1.4_test.csv']
data2 = Data(test_paths = test_data_path)
test_set = CLUTRRdata(data2.test, dictionary)

testloader = DataLoader(dataset = test_set,
                    batch_size = 20,
                    shuffle = True,
                    collate_fn=collate)
evaluate(encoder, decoder, testloader, nn.CrossEntropyLoss())

(tensor(0.2822), 31.57894736842105)